In [1]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

In [2]:
text4000 = pd.read_csv(".//Data/4000/4000-stories-VAD.csv")

In [3]:
text4000.head(1)

,url,length,title,text_no,author,story,valence,arousal,dominance,text_id,...,290,291,292,293,294,295,296,297,298,299
0,https://americanliterature.com/author/eleanor-...,15044,"Peace on Earth, Good-Will to Dogs",0,Eleanor Hallowell Abbott,"PART I\n\nIf you don't like Christmas stories,...",0.592896,0.397839,0.569567,0_eleanor-hallowell-abbott-peace-on-earth-good...,...,3.793141,3.837345,3.778354,-0.815515,-0.72044,-10.738245,-8.765683,0.875089,-10.176691,1.736791


In [4]:
text4000.keys()

Index(['url', 'length', 'title', 'text_no', 'author', 'story', 'valence',
       'arousal', 'dominance', 'text_id',
       ...
       '290', '291', '292', '293', '294', '295', '296', '297', '298', '299'],
      dtype='object', length=317)

In [32]:
lengths = text4000['length'][100:199]
print(lengths)

100       84
101      141
102      141
103      140
104      191
       ...  
194    16642
195      817
196     7148
197    13519
198     5221
Name: length, Length: 99, dtype: int64


In [35]:
text4000['story'][99]

'A Gnat flew over the meadow with much buzzing for so small a creature and settled on the tip of one of the horns of a Bull. After he had rested a short time, he made ready to fly away. But before he left he begged the Bull\'s pardon for having used his horn for a resting place.\n\n"You must be very glad to have me go now," he said.\n\n"It\'s all the same to me," replied the Bull. "I did not even know you were there."\n\nWe are often of greater importance in our own eyes than in the eyes of our neighbor.\n\nThe smaller the mind the greater the conceit.'

In [5]:
texts= list(text4000.story.values)

In [6]:
# Texte mit Trennzeichen oder speziellem Wort kombinieren
trennzeichen = "trennzeichen"
joined_text = (" " + trennzeichen + " ").join(texts)

In [7]:
# Tokenizer erstellen und Texte darauf anwenden
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(joined_text.lower())

In [29]:
context_words = 10
input_words = []
next_words = []
words_limiter = 200000  # limitiert die Anzahl an Trainingsdaten
counter=0
i = 0
while i < len(tokens) - context_words:
    if tokens[i + context_words] == trennzeichen:
        i += context_words + 1
        counter+=1
        continue  # Eintrag überspringen, wenn das Trennzeichen erreicht wird
    
    input_words.append(tokens[i:i + context_words])
    next_words.append(tokens[i + context_words])
    
    if len(next_words) >= words_limiter:
        break
    
    i += 1

print(len(next_words))
print(counter)

200000
199


In [18]:
# überprüfen ob alles ok mit dem Trennzeichen ist
if trennzeichen in tokens:
    print("Alles Gut. Das Trennzeichen ist in tokens enthalten.")
else:
    print("Fehler!!! Trennzeichen nicht in tokens!!!")
if trennzeichen in input_words:
    print("Fehler!!! Das Trennzeichen ist in input_words enthalten.!!!")
else:
    print("Alles Gut.")
if trennzeichen in next_words:
    print("Fehler!!! Das Trennzeichen ist in next_words enthalten.!!!")
else:
    print("Alles Gut.")

Alles Gut. Das Trennzeichen ist in tokens enthalten.
Alles Gut.
Alles Gut.


In [11]:
input_tokens = [token for sublist in input_words for token in sublist]
unique_tokens = np.unique(np.concatenate((input_tokens, next_words))) #alle Token 108282 || in 50000 Token sind 7359 unique
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [12]:
len(unique_tokens)

14353

In [13]:
X = np.zeros((len(input_words), context_words, len(unique_tokens)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [14]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i,j, unique_token_index[word]] = 1
    Y[i, unique_token_index[next_words[i]]] = 1

In [15]:
# Aufteilung der Trainings- und Validierungsdaten
split_index = int(len(X) * 0.95)
x_train, x_val = X[:split_index], X[split_index:]
y_train, y_val = Y[:split_index], Y[split_index:]

In [16]:
with open('FormatedData/200000_fixed/x_train.pickle', 'wb') as file:
    pickle.dump(x_train, file)
with open('FormatedData/200000_fixed/x_val.pickle', 'wb') as file:
    pickle.dump(x_val, file)
with open('FormatedData/200000_fixed/y_train.pickle', 'wb') as file:
    pickle.dump(y_train, file)
with open('FormatedData/200000_fixed/y_val.pickle', 'wb') as file:
    pickle.dump(y_val, file)

In [17]:
with open('FormatedData/200000_fixed/MetaData/context_words.pickle', 'wb') as file:
    pickle.dump(context_words, file)
with open('FormatedData/200000_fixed/MetaData/unique_tokens.pickle', 'wb') as file:
    pickle.dump(unique_tokens, file)
with open('FormatedData/100000_fixed/MetaData/unique_token_index.pickle', 'wb') as file:
    pickle.dump(unique_token_index, file)   